In [18]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

import os
import sys
import pickle
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split 
from sklearn.linear_model import LinearRegression
from sklearn import metrics
import pytz
import ta

import matplotlib.pyplot as plt
import matplotlib as mpl
import seaborn as sns
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.express as px
from matplotlib.dates import DayLocator, HourLocator, DateFormatter, drange
from datetime import datetime
import MetaTrader5 as mt5

from ortisan_ta.dataaccess import DataItem, MetaTraderDataAccess
import ortisan_ta.utils.analysis as ortisan_ta

from ta.volatility import BollingerBands
from ta.trend import ADXIndicator
from ta.volume import OnBalanceVolumeIndicator
from ortisan_ta.simulator import MarketSimulator


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [19]:
#Settings
mpl.style.use('seaborn')
default_fig_size = (20, 20) # Width , Height

### Symbols in my historical database

In [20]:
symbols = ["ABEV3",
"AZUL4",
"B3SA3",
"BBAS3",
"BBDC4",
"BBSE3",
"BPAC11",
"BRDT3",
"BRFS3",
"BRML3",
"BTOW3",
"CCRO3",
"CIEL3",
"CMIG4",
"COGN3",
"CSNA3",
"CVCB3",
"CYRE3",
"ELET3",
"EQTL3",
"GGBR4",
"GNDI3",
"GOAU4",
"GOLL4",
"IRBR3",
"ITSA4",
"ITUB4",
"JBSS3",
"KLBN11",
"LAME4",
"LREN3",
"MGLU3",
"MRFG3",
"MULT3",
"NTCO3",
"PETR3",
"PETR4",
"PRIO3",
"RADL3",
"RAIL3",
"RENT3",
"SBSP3",
"SULA11",
"SUZB3",
"TOTS3",
"UGPA3",
"USIM5",
"VALE3",
"VVAR3",
"WEGE3"]


In [21]:
symbols

['ABEV3',
 'AZUL4',
 'B3SA3',
 'BBAS3',
 'BBDC4',
 'BBSE3',
 'BPAC11',
 'BRDT3',
 'BRFS3',
 'BRML3',
 'BTOW3',
 'CCRO3',
 'CIEL3',
 'CMIG4',
 'COGN3',
 'CSNA3',
 'CVCB3',
 'CYRE3',
 'ELET3',
 'EQTL3',
 'GGBR4',
 'GNDI3',
 'GOAU4',
 'GOLL4',
 'IRBR3',
 'ITSA4',
 'ITUB4',
 'JBSS3',
 'KLBN11',
 'LAME4',
 'LREN3',
 'MGLU3',
 'MRFG3',
 'MULT3',
 'NTCO3',
 'PETR3',
 'PETR4',
 'PRIO3',
 'RADL3',
 'RAIL3',
 'RENT3',
 'SBSP3',
 'SULA11',
 'SUZB3',
 'TOTS3',
 'UGPA3',
 'USIM5',
 'VALE3',
 'VVAR3',
 'WEGE3']

### Gets the data for each symbol

In [22]:
data_access = MetaTraderDataAccess()
dfs = data_access.get_rates_from_symbols(symbols, datetime(2020, 1, 1), datetime(2020, 12, 4), mt5.TIMEFRAME_D1)
dfs

{'ABEV3':                       Open   High    Low  Close  Spread    Volume
 Date                                                             
 2020-01-01 21:00:00  18.45  18.83  18.37  18.78       1  16411300
 2020-01-02 21:00:00  18.54  18.74  18.48  18.52       1  20059000
 2020-01-05 21:00:00  18.45  18.75  18.28  18.60       1  14644300
 2020-01-06 21:00:00  18.63  18.81  18.45  18.64       1  18036600
 2020-01-07 21:00:00  18.67  18.74  18.45  18.55       1  14051700
 ...                    ...    ...    ...    ...     ...       ...
 2020-11-29 21:00:00  14.10  14.18  13.67  13.67       1  41880800
 2020-11-30 21:00:00  13.78  14.42  13.77  14.36       1  26702100
 2020-12-01 21:00:00  14.29  14.60  14.16  14.55       1  18287100
 2020-12-02 21:00:00  14.42  14.53  13.99  14.38       1  32918200
 2020-12-03 21:00:00  14.42  14.49  14.24  14.42       1  18361200
 
 [233 rows x 6 columns],
 'AZUL4':                       Open   High    Low  Close  Spread    Volume
 Date            

### Filtering dfs with data (df is not empty)

In [23]:
dfs_with_data = dict(filter(lambda elem: not elem[1].empty, dfs.items()))

In [24]:
dfs_with_data

{'ABEV3':                       Open   High    Low  Close  Spread    Volume
 Date                                                             
 2020-01-01 21:00:00  18.45  18.83  18.37  18.78       1  16411300
 2020-01-02 21:00:00  18.54  18.74  18.48  18.52       1  20059000
 2020-01-05 21:00:00  18.45  18.75  18.28  18.60       1  14644300
 2020-01-06 21:00:00  18.63  18.81  18.45  18.64       1  18036600
 2020-01-07 21:00:00  18.67  18.74  18.45  18.55       1  14051700
 ...                    ...    ...    ...    ...     ...       ...
 2020-11-29 21:00:00  14.10  14.18  13.67  13.67       1  41880800
 2020-11-30 21:00:00  13.78  14.42  13.77  14.36       1  26702100
 2020-12-01 21:00:00  14.29  14.60  14.16  14.55       1  18287100
 2020-12-02 21:00:00  14.42  14.53  13.99  14.38       1  32918200
 2020-12-03 21:00:00  14.42  14.49  14.24  14.42       1  18361200
 
 [233 rows x 6 columns],
 'AZUL4':                       Open   High    Low  Close  Spread    Volume
 Date            

### Sample

In [25]:
example_asset = "ITUB4"
df = dfs_with_data[example_asset]
df

,Open,High,Low,Close,Spread,Volume
Date,,,,,,
2020-01-01 21:00:00,35.80,36.52,35.53,36.52,1,20666100
2020-01-02 21:00:00,36.02,36.73,35.97,36.14,1,24891400
2020-01-05 21:00:00,36.06,36.09,35.45,35.60,1,22294700
2020-01-06 21:00:00,35.60,35.77,34.78,34.78,1,20000900
2020-01-07 21:00:00,35.01,35.35,34.21,34.21,1,25980900
...,...,...,...,...,...,...
2020-11-29 21:00:00,28.83,29.06,28.43,28.43,1,59889300
2020-11-30 21:00:00,28.93,30.34,28.82,29.63,1,51364700
2020-12-01 21:00:00,29.79,30.11,29.32,29.85,1,32159500


In [26]:
close = df["Close"]
volume = df["Volume"]
high = df["High"]
low = df["Low"]

In [27]:
fig = make_subplots(rows=2, cols=1)

fig.add_trace(go.Scatter(
    x=close.index, y=close, name='Close Price'
), row=1, col=1)

fig.add_trace(go.Scatter(
    x=volume.index, y=volume, name='Volume'
), row=2, col=1)

fig.update_layout(height=1000, width=1000, title_text="Closes and Volume", title_x=0.5)
fig.show()

In [28]:
ema_1 = ortisan_ta.ema(close, 1)
ema_5 = ortisan_ta.ema(close, 5)
ema_20 = ortisan_ta.ema(close, 20)

In [29]:
fig = make_subplots(rows=1, cols=1)

fig.add_trace(go.Scatter(
    x=close.index, y=close, name='Close Price'
), row=1, col=1)

fig.add_trace(go.Scatter(
    x=ema_1.index, y=ema_1, name='Ema 1'
), row=1, col=1)

fig.add_trace(go.Scatter(
    x=ema_5.index, y=ema_5, name='Ema 5'
), row=1, col=1)

fig.add_trace(go.Scatter(
    x=ema_20.index, y=ema_20, name='Ema 20'
), row=1, col=1)

fig.update_layout(height=1000, width=1000, title_text="Closes", title_x=0.5)
fig.show()

### Idenfify pullbacks

In [55]:
pullback_up = (ema_20.shift(3) < ema_20.shift(2)) & (ema_20.shift(1) >= ema_20)

In [56]:
close_pull_up = close[pullback_up]

In [57]:
fig = make_subplots(rows=1, cols=1)

fig.add_trace(go.Scatter(
    x=close.index, y=close, name='Close Price'
), row=1, col=1)

fig.add_trace(go.Scatter(
    x=ema_5.index, y=ema_5, name='EMA 5'
), row=1, col=1)

fig.add_trace(go.Scatter(
    x=ema_20.index, y=ema_20, name='EMA 20'
), row=1, col=1)

fig.add_trace(go.Scatter(
    x=close_pull_up.index, y=close_pull_up, name='Pullback', mode='markers'
), row=1, col=1)

fig.update_layout(height=1000, width=1000, title_text="Closes", title_x=0.5)
fig.show()

In [75]:
# Smooth the line. Remove false signals
roc_1 = ortisan_ta.roc(close, 2)
ema5_roc = ortisan_ta.ema(roc_1, 5)
ema5_roc.head(10)


Date
2020-01-02 21:00:00         NaN
2020-01-05 21:00:00         NaN
2020-01-06 21:00:00         NaN
2020-01-07 21:00:00         NaN
2020-01-08 21:00:00   -1.692936
2020-01-09 21:00:00   -1.426865
2020-01-12 21:00:00   -0.509875
2020-01-13 21:00:00   -0.339917
2020-01-14 21:00:00   -0.632511
2020-01-15 21:00:00   -0.351520
Name: Close, dtype: float64

In [77]:
fig = make_subplots(rows=2, cols=1)

fig.add_trace(go.Scatter(
    x=close.index, y=close, name='Close Price'
), row=1, col=1)

fig.add_trace(go.Scatter(
    x=ema5_roc.index, y=ema5_roc, name='Ema 5 of ROC 1 period'
), row=2, col=1)

fig.update_layout(height=1000, width=1000, title_text="Closes", title_x=0.5)
fig.show()

In [100]:
pullback_up_new = (ema5_roc.shift(3) < 0) & (ema5_roc.shift(2) < 0) & (ema5_roc.shift(1) > 0) & (ema5_roc > 0)

In [101]:
df_pullback = pd.DataFrame({"pullback_up": pullback_up, "close": close})
df_pullback.head()
df_pullback.dropna()

close_pullback = df_pullback[df_pullback.pullback_up]["close"]
close_pullback

Date
2020-02-13 21:00:00    32.49
2020-02-19 21:00:00    32.67
2020-04-08 21:00:00    23.24
2020-04-14 21:00:00    23.14
2020-04-29 21:00:00    22.52
2020-05-14 21:00:00    21.41
2020-05-18 21:00:00    21.47
2020-06-23 21:00:00    25.59
2020-06-25 21:00:00    25.46
2020-06-29 21:00:00    25.20
2020-07-22 21:00:00    26.26
2020-07-30 21:00:00    26.65
2020-08-03 21:00:00    25.47
2020-09-07 21:00:00    24.25
2020-10-15 21:00:00    22.99
2020-10-27 21:00:00    23.65
Name: close, dtype: float64

In [102]:
fig = make_subplots(rows=1, cols=1)

fig.add_trace(go.Scatter(
    x=close.index, y=close, name='Close Price'
), row=1, col=1)

fig.add_trace(go.Scatter(
    x=close_pullback.index, y=close_pullback, name='Pullback UP', mode='markers'
), row=1, col=1)

fig.update_layout(height=1000, width=1000, title_text="Closes", title_x=0.5)
fig.show()